In [1]:
import os.path
import itertools
import numpy as np
import scipy.ndimage as ndi
from skimage import io, color, filters, feature, segmentation, measure, draw, exposure
from sklearn import cluster # scikit-learn: a machine learning library
import matplotlib as mpl
import matplotlib.pylab as plt
import cv2
plt.gray();

<Figure size 432x288 with 0 Axes>

In [3]:
mpl.rcParams['figure.figsize'] = (10,10)

In [2]:
if os.path.isdir('ImageProcessingF2021'):
    !git -C ImageProcessingF2021 pull
else:
    !git clone https://github.com/MoravianCollege/ImageProcessingF2021.git

Cloning into 'ImageProcessingF2021'...
remote: Enumerating objects: 111, done.
remote: Counting objects: 100% (111/111), done.
remote: Compressing objects: 100% (95/95), done.
remote: Total 111 (delta 36), reused 91 (delta 16), pack-reused 0
Receiving objects: 100% (111/111), 10.51 MiB | 42.53 MiB/s, done.
Resolving deltas: 100% (36/36), done.
